## Step 01: Install required librairies

In [1]:
!pip install torch datasets evaluate --upgrade transformers

## Step 02: Check GPU availability

In [2]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("GPU is available:", torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print("GPU not availabl, using CPU")

GPU is available: Tesla T4


## Step 03: Load Dataset

In [4]:
from datasets import load_dataset

ds = load_dataset("google/code_x_glue_ct_code_to_text", "python")

# preview dataset
print(ds)
print("Example entry:")
print(ds['train'][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 14918
    })
})
Example entry:
{'id': 0, 'repo': 'proycon/pynlpl', 'path': 'pynlpl/formats/folia.py', 'func_name': 'AbstractElement.settext', 'original_string': 'def settext(self, text, cls=\'current\'):\n        """Set the text for this element.\n\n        Arguments:\n            text (str): The text\n            cls (str): The class of the text, defaults to ``curren

## Step 04: Load Model & Tokenizer

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Salesforce/codet5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

## Step 05: Preprocess Dataset

Tokenize both **code(input)** and **docstring(labels)**:

In [6]:
max_source_length = 256
max_target_length = 128

def preprocess_function(examples):
  inputs = examples["code"]
  targets = examples["docstring"]

  model_inputs = tokenizer(inputs, max_length = max_source_length, truncation=True)

  labels = tokenizer(targets, max_length=max_target_length, truncation=True)

  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

tokenized_datasets = ds.map(preprocess_function,batched=True)

Map:   0%|          | 0/13914 [00:00<?, ? examples/s]

## Step 06: Set Up Training     

In [22]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import evaluate
import numpy as np

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Evaluation metric
bleu = evaluate.load("bleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Ensure labels are converted to numpy arrays if they are not already
    if not isinstance(labels, np.ndarray):
        labels = labels.numpy()

    # Replace -100 in labels as we can't decode them.
    # We are only interested in the part that was not padded.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Ensure that the lengths of predictions and references match
    # This might be necessary if skip_special_tokens removed tokens from one but not the other
    # In this specific case, we expect them to align after handling -100
    # If misalignment persists, further debugging would be needed

    return bleu.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./codet5-summarizer",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    predict_with_generate=True,
    fp16=True,
    report_to=[],
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(5000)),
    eval_dataset=tokenized_datasets["validation"].select(range(500)),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

## Step 07: Train the Model

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Precisions,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,0.011300,0.048004,0.013353,"[0.9909581218274112, 0.988973121984838, 0.9879336349924586, 0.9866777685262281]",0.013507,0.188517,6304,33440
2,0.019000,0.046482,0.013415,"[0.9912822951339356, 0.9893269065243587, 0.9883217178376342, 0.9871075067581618]",0.013564,0.188666,6309,33440
3,0.016800,0.042820,0.013423,"[0.9911251980982567, 0.9891566265060241, 0.988135593220339, 0.9869022869022869]",0.013575,0.188696,6310,33440


TrainOutput(global_step=1875, training_loss=0.019458872350056965, metrics={'train_runtime': 360.8194, 'train_samples_per_second': 41.572, 'train_steps_per_second': 5.197, 'total_flos': 1008175882174464.0, 'train_loss': 0.019458872350056965, 'epoch': 3.0})

## Step 08: Evaluate the Model

In [24]:
trainer.evaluate()

{'eval_loss': 0.04281972721219063,
 'eval_bleu': 0.013423361221876786,
 'eval_precisions': [0.9911251980982567,
  0.9891566265060241,
  0.988135593220339,
  0.9869022869022869],
 'eval_brevity_penalty': 0.013575011524885359,
 'eval_length_ratio': 0.18869617224880383,
 'eval_translation_length': 6310,
 'eval_reference_length': 33440,
 'eval_runtime': 20.7061,
 'eval_samples_per_second': 24.147,
 'eval_steps_per_second': 3.043,
 'epoch': 3.0}

## Step 09: Inference

In [29]:
# Example code snippet for inference
code_snippet = """
import random

# A list of greetings
greetings = ["Hello", "Hi", "Hey", "Greetings", "Welcome"]

# Get the user's name
name = input("Please enter your name: ")

# A conditional statement to check if the user entered a name
if name:
    # A for loop to print a random greeting five times
    for i in range(5):
        # Select a random greeting from the list
        greeting = random.choice(greetings)
        # Print the personalized greeting
        print(f"{greeting}, {name}!")
else:
    # A different message if no name was entered
    print("Hello there! You didn't enter a name.")

# A final message
print("\nThis is a simple Python script.")
print("It demonstrates lists, loops, conditionals, and user input.")

# End of script
"""

# Tokenize the code snippet
input_ids = tokenizer(code_snippet, return_tensors="pt", max_length=max_source_length, truncation=True).input_ids.to(device)

# Generate the docstring
generated_ids = model.generate(input_ids, max_length=max_target_length)
generated_docstring = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Generated Docstring:", generated_docstring)

Generated Docstring: A list of greetings
greetings = ["Hello", "Hi", "Hey", "Greetings", "Welcome"]

# Get the user's name
name = input("Please enter your name: ")

# A conditional statement to check if the user entered a name
if name:
    # A for loop to print a random greeting five times
    for i in range(5):
        # Select a random greeting from the list
        greeting = random.choice(greetings)
        # Print the personalized greeting
        print
